<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Parallelization-Lab" data-toc-modified-id="Parallelization-Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Parallelization Lab</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Use-the-requests-library-to-retrieve-the-content-from-the-URL-below." data-toc-modified-id="Step-1:-Use-the-requests-library-to-retrieve-the-content-from-the-URL-below.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Step 1: Use the requests library to retrieve the content from the URL below.</a></span></li><li><span><a href="#Step-2:-Use-BeautifulSoup-to-extract-a-list-of-all-the-unique-links-on-the-page." data-toc-modified-id="Step-2:-Use-BeautifulSoup-to-extract-a-list-of-all-the-unique-links-on-the-page.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.</a></span></li><li><span><a href="#Step-3:-Use-list-comprehensions-with-conditions-to-clean-the-link-list." data-toc-modified-id="Step-3:-Use-list-comprehensions-with-conditions-to-clean-the-link-list.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Step 3: Use list comprehensions with conditions to clean the link list.</a></span></li><li><span><a href="#Step-4:-Use-the-os-library-to-create-a-folder-called-wikipedia-and-make-that-the-current-working-directory." data-toc-modified-id="Step-4:-Use-the-os-library-to-create-a-folder-called-wikipedia-and-make-that-the-current-working-directory.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Step 4: Use the os library to create a folder called <em>wikipedia</em> and make that the current working directory.</a></span></li><li><span><a href="#Step-5:-Write-a-function-called-index_page-that-accepts-a-link-and-does-the-following." data-toc-modified-id="Step-5:-Write-a-function-called-index_page-that-accepts-a-link-and-does-the-following.-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Step 5: Write a function called index_page that accepts a link and does the following.</a></span></li><li><span><a href="#Step-6:-Sequentially-loop-through-the-list-of-links,-running-the-index_page-function-each-time." data-toc-modified-id="Step-6:-Sequentially-loop-through-the-list-of-links,-running-the-index_page-function-each-time.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Step 6: Sequentially loop through the list of links, running the index_page function each time.</a></span></li><li><span><a href="#Step-7:-Perform-the-page-indexing-in-parallel-and-note-the-difference-in-performance." data-toc-modified-id="Step-7:-Perform-the-page-indexing-in-parallel-and-note-the-difference-in-performance.-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Step 7: Perform the page indexing in parallel and note the difference in performance.</a></span></li></ul></li></ul></div>

# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

https://en.wikipedia.org/wiki/Data_science

In [4]:
import requests as req

url = 'https://en.wikipedia.org/wiki/Data_science'

In [7]:
req.get(url).content[:500]

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-language-alert-in-sidebar-enabled vector-feature-sticky-header-disabled vector-feature-page-tools-disabled vector-feature-page-tools-pinned-disabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Data science - Wi'

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [8]:
from bs4 import BeautifulSoup as bs
import re

In [10]:
html = req.get(url).content

In [11]:
soup = bs(html, 'html.parser')

In [50]:
link =soup.find_all('a', href = True)

lst = []

for i in link:
    lst.append(i.get('href'))

lst[:10]

['#bodyContent',
 '/wiki/Main_Page',
 '/wiki/Special:Search',
 '/w/index.php?title=Special:CreateAccount&returnto=Data+science',
 '/w/index.php?title=Special:CreateAccount&returnto=Data+science',
 '/w/index.php?title=Special:UserLogin&returnto=Data+science',
 '/wiki/Help:Introduction',
 '/wiki/Special:MyTalk',
 '/wiki/Special:MyContributions',
 '/wiki/Main_Page']

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [45]:
domain = 'http://wikipedia.org'

In [49]:
absolute = []
relative = []

for i in lst:
    if i.startswith('http'):
        absolute.append(i)
    elif i.startswith('/wiki'):
        relative.append(domain + i)
    else:
        continue
abs_and_rel = absolute + relative
abs_and_rel[:10]      

['https://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en',
 'https://www.wikidata.org/wiki/Special:EntityPage/Q2374463',
 'https://commons.wikimedia.org/wiki/Category:Data_science',
 'https://en.wikiversity.org/wiki/Data_science',
 'https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_%D8%A7%D9%84%D8%A8%D9%8A%D8%A7%D9%86%D8%A7%D8%AA',
 'https://az.wikipedia.org/wiki/Veril%C9%99nl%C9%99r_elmi_(Data_Science)',
 'https://bn.wikipedia.org/wiki/%E0%A6%89%E0%A6%AA%E0%A6%BE%E0%A6%A4%E0%A7%8D%E0%A6%A4_%E0%A6%AC%E0%A6%BF%E0%A6%9C%E0%A7%8D%E0%A6%9E%E0%A6%BE%E0%A6%A8',
 'https://bg.wikipedia.org/wiki/%D0%9D%D0%B0%D1%83%D0%BA%D0%B0_%D0%B7%D0%B0_%D0%B4%D0%B0%D0%BD%D0%BD%D0%B8%D1%82%D0%B5',
 'https://ca.wikipedia.org/wiki/Ci%C3%A8ncia_de_les_dades',
 'https://cs.wikipedia.org/wiki/Data_science']

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [51]:
import os

In [63]:
os.chdir('wikipedia')

In [64]:
print(os.getcwd())

C:\Users\calba\OneDrive\Desktop\4.4-lab-parallelization\your-code\wikipedia


## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [65]:
pip install python-slugify

Note: you may need to restart the kernel to use updated packages.


In [68]:
from slugify import slugify

In [114]:
def index_page(link):
    
    try:
        nombre=slugify(link)
        slug = slugify(req.get(link).content)
        f = open(f'{nombre}.html', "w")
        f.write(slug)
    except:
        pass
    
    return

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [110]:
from tqdm.notebook import tqdm

In [115]:
%%time

for i in tqdm(abs_and_rel):
    index_page(i)

  0%|          | 0/305 [00:00<?, ?it/s]

Wall time: 3min 50s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [117]:
import multiprocessing as mp

In [125]:
import funcion
from funcion import index_page

In [118]:
mp.get_start_method()

'spawn'

In [119]:
from multiprocessing import get_context

In [120]:
mp.cpu_count()

8

In [126]:
%%time

pool=get_context().Pool(6)  # grupo con 6 cores

res=pool.map_async(funcion.index_page, tqdm(abs_and_rel)).get()
pool.close()

  0%|          | 0/305 [00:00<?, ?it/s]

Wall time: 44.2 s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,